This notebook directly works with implementing and producing results, of chosen metrics, correlated with this project

## Metric #1: CLIP-Based Style Similarity

### Given

* A fixed evaluation prompt set divided into:
  * Positive prompts (intended to elicit the tattoo style)
  * Negative prompts (intended to avoid the tattoo style)

* A fixed set of random seeds

* Three model configurations:
  * Base model
  * Base + Original LoRA (fused)
  * Base + Recreated LoRA (fused)

* A held-out dataset of tattoo-style reference images not used during LoRA training

All generation parameters (sampler, number of steps, guidance scale, resolution) are held constant across models.

---

### Procedure

1. For each model, generate images using the same prompts and seeds.

2. Compute CLIP image embeddings for:
   * All generated images
   * All held-out reference tattoo images

3. For each generated image:
   * Compute cosine similarity to the reference set
   * Use top-k average similarity (or nearest-neighbor similarity)

4. Aggregate similarity scores:
   * Mean and standard deviation across all images
   * Separate averages for positive prompts
   * Separate averages for negative prompts

5. Optionally compute a normalized recovery score:
   * Recovery = (Score_recreated − Score_base) / (Score_original − Score_base)

---

### Output

For each model, report:

* Mean CLIP similarity to held-out tattoo references
* Standard deviation
* Separate similarity scores for positive and negative prompts
* Optional: normalized recovery score

---

### Meaning

This metric evaluates how closely generated images align with unseen tattoo-style works in CLIP embedding space.

If the recreated model achieves similarity scores close to the original LoRA model—and substantially higher than the base model—this indicates successful recovery of stylistic behavior.

A strong reconstruction should:

* Produce high similarity for positive prompts
* Maintain low similarity for negative prompts
* Closely match the original LoRA’s embedding-level behavior


### Load base, base+fused, base+recovered models

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, AutoencoderKL

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.float16 if device == "cuda" else torch.float32

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=dtype)

def load_pipe(repo_id: str): # helper function to load the models modularly
    pipe = StableDiffusionXLPipeline.from_pretrained(
        repo_id,
        vae=vae,
        torch_dtype=dtype,
        variant="fp16" if dtype == torch.float16 else None,
        use_safetensors=True,
    ).to(device)
    pipe.set_progress_bar_config(disable=True)
    return pipe

pipe_base  = load_pipe("stabilityai/stable-diffusion-xl-base-1.0") # base
pipe_fused = load_pipe("francuzama/sdxl-fused") # original+fused
pipe_recovered = load_pipe("francuzama/sdxl-recovered") # original+recovered

positive_prompt_set = ["a tattoo in the style of JASON, a roaring tiger, bold black ink, yellow highlights, clean linework, high contrast", "a tattoo in the style of JASON, a dagger wrapped in roses, bold black ink, yellow highlights, sharp outlines, flat shadin", "a tattoo in the style of JASON, a mechanical owl with spread wings, bold black ink, yellow highlights, strong outlines, graphic style"]
negative_prompt_set = ["a roaring tiger, realistic wildlife photograph, natural lighting, DSLR photo", "a dagger wrapped in roses, watercolor illustration, soft edges, pastel colors", "a mechanical owl with spread wings, 3D render, cinematic lighting, ultra detailed"]
seed_set = [1234, 667, 3067]

def gen(pipe, outname):
    g = torch.Generator(device=device).manual_seed(seed)
    img = pipe(prompt, num_inference_steps=30, guidance_scale=7.0, generator=g).images[0]
    img.save(outname)

gen(pipe_base,  "base.png")
gen(pipe_fused, "fused.png")

print("Saved base.png and fused.png")


## Metric #2: Weight-Space Reconstruction Similarity

### Given

* Three model checkpoints:
  * Base model  
  * Base + Original LoRA (fused)  
  * Base + Recreated LoRA (fused)

* Access to full model weights (e.g., UNet and relevant submodules)

* The ability to compute weight differences relative to the base model

All models must share the same architecture and base initialization.

---

### Procedure

1. Compute weight deltas relative to the base model:

   * Delta (Original) = Fused Original − Base  
   * Delta (Recreated) = Fused Recreated − Base  

2. For each corresponding layer:

   * Flatten weight tensors  
   * Compute L2 distance between delta tensors  
   * Optionally compute cosine similarity between flattened tensors  

3. Aggregate across layers:

   * Compute overall L2 distance  
   * Compute relative L2 error:
     * Relative Error = ||Delta_original − Delta_recreated||₂ / ||Delta_original||₂  

4. Optionally analyze similarity at different levels:

   * Per-layer similarity (attention blocks, MLP layers, etc.)  
   * Global similarity across entire UNet  

---

### Output

For the reconstructed model, report:

* Overall L2 distance between weight deltas  
* Relative L2 error  
* Optional: cosine similarity between deltas  
* Optional: per-layer similarity statistics (mean, worst layer, etc.)

---

### Meaning

This metric evaluates how closely the attacker’s reconstructed model matches the original fused model in parameter space.

If the recreated delta closely matches the original delta (low relative L2 error, high cosine similarity), this indicates successful recovery of the learned style at the parameter level.

A strong reconstruction should:

* Produce small global L2 difference  
* Maintain high cosine alignment between weight deltas  
* Show consistent similarity across major architectural components
